In [1]:
spark.version

'3.0.1'

In [2]:
import subprocess

from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [3]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [4]:
directory = 'gs://folder-name'
# file = '*.json'
# path = directory + file
path = directory

In [5]:
cmd = 'hadoop fs -du -s -h ' + directory

p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
for line in p.stdout.readlines():
    print (line)
    
retval = p.wait()

156.2 G  156.2 G  gs://msca-bdp-tweets/final_project



In [6]:
%%time

tweets_df = spark.read.json(path)

CPU times: user 38.1 ms, sys: 8.58 ms, total: 46.7 ms
Wall time: 3min 58s


In [26]:
data = tweets_df.select([tweets_df.text,
                         tweets_df.id,
                         tweets_df.retweeted_status,
                         tweets_df.user['name'].alias("user_name"), 
                         tweets_df.user['description'].alias("user_description"),
                         tweets_df.user.verified.alias("verified_user"), 
                         tweets_df.user['followers_count'].alias("followers_count")])

In [27]:
#convert columns with strings that I will use to lowercase
import pandas as pd

import pyspark.sql.functions as f

data = data\
    .withColumn("user_name",f.lower(f.col("user_name")))\
    .withColumn("user_description",f.lower(f.col("user_description")))\
    .withColumn("text",f.lower(f.col("text")))

In [28]:
#create dataframes with strings with covid-related terms
covid_data2 = data.filter((data.text.contains("covid"))\
                          | (data.text.contains("coronavirus"))\
                          | (data.text.contains("pandemic"))\
                          | (data.text.contains("vaccin"))\
                          | (data.text.contains("booster"))\
                          | (data.text.contains("pfizer"))\
                          | (data.text.contains("moderna"))\
                          | (data.text.contains("j&j"))\
                          | (data.text.contains("johnson and johnson")))

In [29]:
covid_data2

text,id,retweeted_status,user_name,user_description,verified_user,followers_count
@garraarghhrumph ...,1454279983531188226,null,emily joy,mostly skating ba...,false,70
rt @denise_old_la...,1454279984017600512,"[,, Sat Oct 30 02...",judith becker,soldier of the lion,false,919
@pahpcorn @lisa_t...,1454279984198021126,null,therealfourseason...,often tweets abou...,false,1869
@polan13 @mark_do...,1454279984466386954,null,aaron bradley,futures trader. i...,false,650
rt @legendaryener...,1454279984722419712,"[,, Fri Oct 29 17...",bright starlight,null,false,359
rt @rwmalonemd: h...,1454279985292840962,"[,, Fri Oct 29 16...",toby zimmerman 🙂,null,false,87
rt @choi_bts2: it...,1454279985439522822,"[,, Fri Oct 29 23...",wadaaa | fucking ...,be a crave of pea...,false,129
rt @lukewearechan...,1454279985519288323,"[,, Sat Oct 30 01...",akira morgendorffer,chill. spunky. aw...,false,133
while one can sti...,1454279987012390913,null,onkar gharat,"works @ #spmrm, m...",false,100
rt @ctvnews: edmo...,1454279987289153536,"[,, Sat Oct 30 01...",robert rgyatso,add appropriate c...,false,557


In [30]:
data_df2 = covid_data2

In [31]:
data_df3 = data_df2.withColumn("Organization", 
                        f.when(data_df2.verified_user == 'false', 'Other') \
                        .when((data_df2.user_description.contains('health')) | (data_df2.user_name.contains('health')) \
                              | (data_df2.user_description.contains('medic')) | (data_df2.user_name.contains('medic')) \
                              | (data_df2.user_description.contains('clinic')) | (data_df2.user_name.contains('clinic')) \
                              | (data_df2.user_description.contains('nurse')) | (data_df2.user_name.contains('nurse')) \
                              | (data_df2.user_description.contains('doctor')) | (data_df2.user_name.contains('doctor')) \
                              | (data_df2.user_description.contains('hospital')) | (data_df2.user_name.contains('hospital')) \
                              | (data_df2.user_description.contains('disease')) | (data_df2.user_name.contains('disease')), 'Health') \
                        .when((data_df2.user_description.contains('gov')) | (data_df2.user_name.contains('gov')) \
                              | (data_df2.user_description.contains('president')) | (data_df2.user_name.contains('president')) \
                              | (data_df2.user_description.contains('senat')) | (data_df2.user_name.contains('senat')) \
                              | (data_df2.user_description.contains('congress')) | (data_df2.user_name.contains('congress')) \
                              | (data_df2.user_description.contains('prime minister')) | (data_df2.user_name.contains('prime minister')) \
                              | (data_df2.user_description.contains('parliament')) | (data_df2.user_name.contains('parliament')) \
                              | (data_df2.user_description.contains('vp')) | (data_df2.user_name.contains('vp')) \
                              | (data_df2.user_description.contains('white house')) | (data_df2.user_name.contains('white house')) \
                              | (data_df2.user_description.contains('mayor')) | (data_df2.user_name.contains('mayor')) \
                              | (data_df2.user_description.contains('democr')) | (data_df2.user_name.contains('democr')) \
                              | (data_df2.user_description.contains('republic')) | (data_df2.user_name.contains('republic')), 'Government') \
                        .when((data_df2.user_description.contains('news')) | (data_df2.user_name.contains('news')) \
                              | (data_df2.user_description.contains('daily')) | (data_df2.user_name.contains('daily')) \
                              | (data_df2.user_description.contains('times')) | (data_df2.user_name.contains('times')) \
                              | (data_df2.user_description.contains('journal')) | (data_df2.user_name.contains('journal')), 'News') \
                        .when(((data_df2.user_description.contains('actor')) & (data_df2.followers_count > 30000)) \
                              | ((data_df2.user_description.contains('singer')) & (data_df2.followers_count > 30000)) \
                              | ((data_df2.user_description.contains('writer')) & (data_df2.followers_count > 30000)) \
                              | ((data_df2.user_description.contains('play')) & (data_df2.followers_count > 30000)) \
                              | ((data_df2.user_description.contains('musician')) & (data_df2.followers_count > 30000)) \
                              | ((data_df2.user_description.contains('actress')) & (data_df2.followers_count > 30000)) \
                              | ((data_df2.user_description.contains('golfer')) & (data_df2.followers_count > 30000)) \
                              | ((data_df2.user_description.contains('author')) & (data_df2.followers_count > 30000)), 'Celebrity') \
                        .when(data_df2.followers_count >= 10000, 'Influencer')) 

In [32]:
data_df4 = data_df3.na.fill(value='Other',subset=["Organization"])

In [33]:
data_df4

text,id,retweeted_status,user_name,user_description,verified_user,followers_count,Organization
@garraarghhrumph ...,1454279983531188226,null,emily joy,mostly skating ba...,false,70,Other
rt @denise_old_la...,1454279984017600512,"[,, Sat Oct 30 02...",judith becker,soldier of the lion,false,919,Other
@pahpcorn @lisa_t...,1454279984198021126,null,therealfourseason...,often tweets abou...,false,1869,Other
@polan13 @mark_do...,1454279984466386954,null,aaron bradley,futures trader. i...,false,650,Other
rt @legendaryener...,1454279984722419712,"[,, Fri Oct 29 17...",bright starlight,null,false,359,Other
rt @rwmalonemd: h...,1454279985292840962,"[,, Fri Oct 29 16...",toby zimmerman 🙂,null,false,87,Other
rt @choi_bts2: it...,1454279985439522822,"[,, Fri Oct 29 23...",wadaaa | fucking ...,be a crave of pea...,false,129,Other
rt @lukewearechan...,1454279985519288323,"[,, Sat Oct 30 01...",akira morgendorffer,chill. spunky. aw...,false,133,Other
while one can sti...,1454279987012390913,null,onkar gharat,"works @ #spmrm, m...",false,100,Other
rt @ctvnews: edmo...,1454279987289153536,"[,, Sat Oct 30 01...",robert rgyatso,add appropriate c...,false,557,Other


In [34]:
lsh = data_df4.select(['text', 'Organization', 'id', 'retweeted_status'])

In [35]:
lsh

text,Organization,id,retweeted_status
@garraarghhrumph ...,Other,1454279983531188226,null
rt @denise_old_la...,Other,1454279984017600512,"[,, Sat Oct 30 02..."
@pahpcorn @lisa_t...,Other,1454279984198021126,null
@polan13 @mark_do...,Other,1454279984466386954,null
rt @legendaryener...,Other,1454279984722419712,"[,, Fri Oct 29 17..."
rt @rwmalonemd: h...,Other,1454279985292840962,"[,, Fri Oct 29 16..."
rt @choi_bts2: it...,Other,1454279985439522822,"[,, Fri Oct 29 23..."
rt @lukewearechan...,Other,1454279985519288323,"[,, Sat Oct 30 01..."
while one can sti...,Other,1454279987012390913,null
rt @ctvnews: edmo...,Other,1454279987289153536,"[,, Sat Oct 30 01..."


In [ ]:
lsh.write.format("parquet")\
.mode('overwrite')\
.save('gs://folder-name')

print('done')

done
